# Lambda 1 - Serialize Image

In [1]:
import sagemaker
import boto3
import os


bucket= sagemaker.session.Session().default_bucket()
role = sagemaker.get_execution_role()
region = boto3.client('sagemaker').meta.region_name

os.environ["DEFAULT_S3_BUCKET"] = bucket
os.environ["PREFIX"] = 'Udacity-AWSML-PRO2'

prefix = os.environ["PREFIX"]

In [2]:
event = {
  "inferences": [],
  "s3_key": "Udacity-AWSML-PRO2/test/bike_s_002116.png",
  "s3_bucket": "sagemaker-us-east-1-254050731868",
  "image_data": ""
}

In [3]:
import json
import base64
import os
import boto3

def lambda_handler(event, context):
    """A function to serialize target data from S3"""

    s3 = boto3.client('s3')

    # Get the s3 address from the Step Function event input
    key = event['s3_key']
    bucket = event['s3_bucket']
    
    # Download the data from s3 to /tmp/image.png
    s3.download_file(bucket, key, '/tmp/image.png')
    
    with open("/tmp/image.png", "rb") as f:
        image_data = base64.b64encode(f.read())

    # Pass the data back to the Step Function
    print("Event:", event.keys())
    return {
        'statusCode': 200,
        'body': {
            "image_data": image_data,
            "s3_bucket": bucket,
            "s3_key": key,
            "inferences": []
        }
    }

In [4]:
lambda_handler(event, None)

Event: dict_keys(['inferences', 's3_key', 's3_bucket', 'image_data'])


{'statusCode': 200,
 'body': {'image_data': b'iVBORw0KGgoAAAANSUhEUgAAACAAAAAgCAYAAABzenr0AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8/fFQqAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAIzklEQVR4nF2X23Ncx3HGfz0z55zdxZUXiBAhSpEsS5QvEmNVKS4lqfghfrD/2bzEVSlXEpdLTvlu07qBZYaSCBIgAHKxwGIv5zLTnYc5C0t5OzNzpm/fdPfX8rP/+MhGO3tMFpE2GVGELiniHCIQ1UiAE8EAENQUs7xyIiACAGaY5aWIYKq0XSSaYObQpKgmLCZeWq84ffQJ4fDwkL2NHdooqHNENWJKeAwRodeKmWHkpayUAWqr//rz3gjDwEANTEHV0GSoKYVzzC8u+PnP/o2wSI4mGkUR6JLiMLz3iIBzDgxiSjkiZM8ATJXe77wnggCqipqSesPEOcRWdwETgvNMx2f89qNfEF5/5x6Iw3tByIJMNXsqkg0xl2/3YRcRvPe9t3YVofy/4JxHeqiUPmTOwARBMYTpbAoxEhShcOCFXmjCyHgn7UMpkhWJ4HolfM37lRGrtRNBTfr4GypG7xvOAOeZzhY01hBSSnhxiPN0qnRq5OBmJWqGkT2VrOEbSr+ufLVWy/cyHIYhiIEYoKAqzGdzyvI6oVZBKEgp0SYlGhm/K4H01l/tXIX+66pX56q62lkdIPRGYyAexEj1hPUKQqOO1CnBkb03yS97lQWyEiXfFLz6Wj3KVTScoJqtFhEc+eHlzABz+S1dH9W8/7YSmmh0qcN734uUrNyt1K4sz6nkkSs4/pZ2hvV1Qlf1YgWNGWKwuqGWkFQwr2e0g5Lg/h+m+SU7VgXABFRcTma0LzQOE5cV9nuevB8BkwSkXqWhKK4PpYkQVdk/nv

###### Helper Function

In [ ]:
def upload_file_to_s3(file_name, bucket, s3_prefix):
    object_name = os.path.join(s3_prefix, file_name)
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False

##### 1.1 - Serialize Image data

In [ ]:
%%writefile lambda_function.py

# This cell will write the function to your local machine. Note the name of the file and the name of the function. 
# Compare this to the 'Handler' parameter. 

import json
import base64
import os
import boto3

# Lambda Event Structure
#{
#    "inferences": [], # Output of predictor.predict
#    "s3_key": "", # Source data S3 key
#    "s3_bucket": "", # Source data S3 bucket
#    "image_data": ""  # base64 encoded string containing the image data
#}

def lambda_handler(event, context):
    """A function to serialize target data from S3"""

    s3 = boto3.client('s3')

    # Get the s3 address from the Step Function event input
    key = event['s3_key']
    bucket = event['s3_bucket']
    
    # Download the data from s3 to /tmp/image.png
    s3.download_file(bucket, key, '/tmp/image.png')
    
    with open("/tmp/image.png", "rb") as f:
        image_data = base64.b64encode(f.read())

    # Pass the data back to the Step Function
    print("Event:", event.keys())
    return {
        'statusCode': 200,
        'body': {
            "image_data": image_data,
            "s3_bucket": bucket,
            "s3_key": key,
            "inferences": []
        }
    }


###### 1.2 - Zip to a File

In [ ]:
from zipfile import ZipFile

with ZipFile('code.zip', 'w') as f:
    f.write('lambda_function.py')

###### 1.3 - Upload to S3

In [ ]:
#upload_file_to_s3(file_name='code.zip', bucket=bucket, s3_prefix=prefix)

#### 1.4 - Create Lambda Function over AWS

In [ ]:
lambda_client = boto3.client('lambda')

with open('code.zip', 'rb') as f:
    b_code = f.read()

response = lambda_client.create_function(
    FunctionName='serializeImageData',
    Runtime='python3.8',
    Handler='lambda_function.lambda_handler',
    Code={'ZipFile': b_code, },
    Description='string',
    Timeout=30,
    MemorySize=1024,
    Publish=True,
    PackageType='Zip',
    Role=role
)

#### 1.5 - Invoke Endpoint

In [5]:
event = {
  "inferences": [],
  "s3_key": "Udacity-AWSML-PRO2/test/bike_s_002116.png",
  "s3_bucket": "sagemaker-us-east-1-254050731868",
  "image_data": ""
}

In [6]:
import json

lambda_client = boto3.client('lambda')
payload = event

payload_bytes = json.dumps(payload).encode('utf-8')
response = lambda_client.invoke(FunctionName='serializeImageData', InvocationType='Event', Payload=payload_bytes)

In [7]:
response

{'ResponseMetadata': {'RequestId': 'ccea4c7b-226e-43a5-a28f-cdeb95538c53',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 18 Sep 2022 01:02:04 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ccea4c7b-226e-43a5-a28f-cdeb95538c53',
   'x-amzn-remapped-content-length': '0',
   'x-amzn-trace-id': 'root=1-63266e0c-5f4299c14b031a0c70fbd534;sampled=0'},
  'RetryAttempts': 0},
 'StatusCode': 202,
 'Payload': <botocore.response.StreamingBody at 0x7fdd580f5670>}